# Lab | Summarization evaluation using LangSmith
Let's revisit your capstone project 2? Well, sort of. Pick diffierent sets of data and re-run this notebook. Maybe parts of the dataset you used in your last project week. The point is for you to understand all steps involve and the many different ways one can and should evaluate LLM applications using LangSmith.

What did you learn? - Let's discuss that in class

## LangSmith - LangChain evaluation

In [2]:
from google.colab import userdata

OPENAI_API_KEY  = userdata.get('OPENAI_API_KEY')
LANGCHAIN_API_KEY = userdata.get("LANGCHAIN_API_KEY")
HUGGINGFACEHUB_API_TOKEN = userdata.get("HF_TOKEN")

In [4]:
import os

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="langsmith_max-test"

In [6]:
#Importing Client from Langsmith
from langsmith import Client
client = Client(api_key=LANGCHAIN_API_KEY)

### Create Dataset


In [15]:
from datasets import load_dataset
# Load a different summarization dataset, for example 'cnn_dailymail'
cnn_dataset = load_dataset("cnn_dailymail", "3.0.0", split="train")

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [16]:
def add_prefix(example):
    return {
        **example,
        "article": f"Summarize this news:\n{example['article']}"
    }

#cnn_dataset = cnn_dataset.map(add_prefix)

In [17]:
cnn_dataset

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 287113
})

In [19]:
cnn_dataset[0]

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [21]:
#Get just a few news to test
MAX_NEWS=10
test_cnn_dataset = load_dataset("cnn_dailymail", "3.0.0", split="test")
sample_cnn = test_cnn_dataset.select(range(MAX_NEWS)).map(add_prefix)

sample_cnn

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 10
})

The dataset contains three columns: article, highlights, and id. To use LangSmith, we need to create a dataset in LangSmith format.

LangSmith expects a prompt and a result. To achieve this, we will transform the article into a prompt by adding the prefix: "Summarize this news." As a result, we will use the content of highlights, which represents the summaries created by humans.

In [22]:
print(sample_cnn[0])

{'article': 'Summarize this news:\n(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians\' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki,

Now We have the Dataset with the prompt and the Reference Summary, it is time to create a Dataset in LangSmith with this information.
### Create the Dataset in Langsmith

The dataset in LangSmith is composed of an input, which is the prompt passed to the model for evaluation, and an output, which should contain what we expect the model to return.

In [23]:
import datetime

In [24]:
import uuid
input_key=['article']
output_key=['highlights']

NAME_DATASET=f"Summarize_dataset_{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

In [25]:
#This creates the dataset in LangSmith with the content in sample_cnn - If you run this more than once you will get POST errors
dataset = client.upload_dataframe(
    df=sample_cnn,
    input_keys=input_key,
    output_keys=output_key,
    name=NAME_DATASET,
    description="Test Embedding distance between model summarizations",
    data_type="kv"
)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In this image, we can see an example from the dataset once it's been registered in LangSmith.

In the Input column, there is the prompt to be sent, while in the Output column, the expected output is stored.

When performing the comparison, the model will be given the prompt, and the Cosine distance between its response and the one stored in the sample dataset will be calculated.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Dataset.jpg?raw=true">

### Recovering Models From Hugging Face
Let's retrieve both models from HuggingFace. A base T5 model and a model that has been fine-tuned using the training portion of this same dataset to generate summaries.

In [29]:
from langchain_community.llms import HuggingFaceHub

In [27]:
%pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [30]:
summarizer_base = HuggingFaceHub(
    repo_id="t5-base",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

/tmp/ipython-input-3938980585.py:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  summarizer_base = HuggingFaceHub(


In [31]:
summarizer_finetuned = HuggingFaceHub(
    repo_id="flax-community/t5-base-cnn-dm",
    model_kwargs={"temperature":0, "max_length":180},
    huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN
)

## Defining Evaluator
The first step is to define an evaluator, where we specify the variables we want to evaluate. In our case, I have chosen to measure only the "embedding_distance."

I've left the "string_distance" as a comment in case you want to conduct a test with two evaluations instead of one.


In [32]:
from langchain.smith import run_on_dataset, RunEvalConfig
# !pip install -q rapidfuzz==3.6.1

In [33]:
#We are using just one of the multiple evaluator avaiable on LangSmith.

evaluation_config = RunEvalConfig(
    evaluators=[
        "embedding_distance",
        #"string_distance"
    ],
)



### Running Evaluator
With the same configuration, we can launch two evaluations on the same dataset. One for each of the chosen models.

In [35]:
project_name = f"T5-BASE {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Set the OpenAI API key as an environment variable for the evaluator
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

base_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_base,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-BASE 2025-09-09 13:59:48' at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc/compare?selectedSessions=7e4675dc-ec64-4422-b3e4-6f589dde469e

View all tests for Dataset Summarize_dataset_2025-09-09 13:57:19 at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc
[>                                                 ] 0/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------->                                   ] 3/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------->                              ] 4/10

...

[---------------------------------->               ] 7/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------------------------------------->     ] 9/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------------------------------------->] 10/10


In [36]:
#Ignore the error shown below
project_name = f"T5-FineTuned {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=summarizer_finetuned,
    evaluation=evaluation_config,
)

View the evaluation results for project 'T5-FineTuned 2025-09-09 14:00:26' at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc/compare?selectedSessions=ced5d0a8-fe9c-431e-aed9-e21539ccf272

View all tests for Dataset Summarize_dataset_2025-09-09 13:57:19 at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc
[>                                                 ] 0/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------->                                   ] 3/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------->                              ] 4/10

...

[---------------------------------->               ] 7/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'
Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[-------------------------------------------->     ] 9/10

Error Type: AttributeError, Message: 'InferenceClient' object has no attribute 'post'


[------------------------------------------------->] 10/10


<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_Tests.jpg?raw=true">

In the image below you can see the comparision between two tests.
<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareTestst.jpg?raw=true">

Well, since it has been so straightforward, why don't we try to make the comparison with an OpenAI model?

In [40]:
from langchain_openai import OpenAI
open_aillm=OpenAI(temperature=0.0)

In [38]:
%pip install --upgrade --quiet  langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 1.2 MB/s eta 0:00:00


In [41]:
project_name = f"OpenAI {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

finetuned_t5_results = run_on_dataset(
    client=client,
    project_name=project_name,
    dataset_name=NAME_DATASET,
    llm_or_chain_factory=open_aillm,
    evaluation=evaluation_config,
)

View the evaluation results for project 'OpenAI 2025-09-09 14:03:24' at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc/compare?selectedSessions=2bc2c650-9836-4aeb-b3da-6942d9d81987

View all tests for Dataset Summarize_dataset_2025-09-09 13:57:19 at:
https://smith.langchain.com/o/5f427474-b4fb-4849-9cea-6d85c50751b4/datasets/4335033e-cbef-40da-b502-64750306aabc
[------------------------------------------------->] 10/10


<img src="https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_4_2SDL_CompareOpenAI_HF.jpg?raw=true">

The experiment with the OpenAI model has yielded the best results. But, be aware! As we can see, there is a cost involved since we are using an API, and it needs to be paid for.

Another crucial piece of information is that we can view performance data for the models. This data could also be useful for minimally evaluating our inference server.